# Sequential Simulations with [PowerSimulations.jl](https://github.com/NREL-SIIP/PowerSimulations.jl)

**Originally Contributed by**: Clayton Barrows

## Introduction

PowerSimulations.jl supports simulations that consist of sequential optimization problems
where results from previous problems inform subsequent problems in a variety of ways. This
example demonstrates some of these capabilities to represent electricity market clearing.

## Dependencies
Since the `OperatiotnsProblem` is the fundamental building block of a sequential
simulation in PowerSimulations, we will build on the [OperationsProblem example](../../notebook/PowerSimulations_examples/1_operations_problems.ipynb)
by sourcing it as a dependency.

In [1]:
using SIIPExamples
pkgpath = dirname(dirname(pathof(SIIPExamples)))
include(joinpath(pkgpath, "test", "PowerSimulations_examples", "1_operations_problems.jl"))

┌ Info: Parsing csv data in branch.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/m5IPy/src/parsers/power_system_table_data.jl:148
┌ Info: Successfully parsed branch.csv
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/m5IPy/src/parsers/power_system_table_data.jl:153
┌ Info: Parsing csv data in bus.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/m5IPy/src/parsers/power_system_table_data.jl:148
┌ Info: Successfully parsed bus.csv
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/m5IPy/src/parsers/power_system_table_data.jl:153
┌ Info: Parsing csv data in dc_branch.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/m5IPy/src/parsers/power_system_table_data.jl:148
┌ Info: Successfully parsed dc_branch.csv
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/m5IPy/src/parsers/power_system_table_data.jl:153
┌ Info: Parsing csv data in gen.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages


Operations Problem Specification

  transmission:  PowerSimulations.CopperPlatePowerModel
  devices: 
      ILoads:
        device_type = PowerSystems.InterruptibleLoad
        formulation = PowerSimulations.InterruptiblePowerLoad
      HydroROR:
        device_type = PowerSystems.HydroDispatch
        formulation = PowerSimulations.HydroFixed
      Generators:
        device_type = PowerSystems.ThermalStandard
        formulation = PowerSimulations.ThermalStandardUnitCommitment
      Ren:
        device_type = PowerSystems.RenewableDispatch
        formulation = PowerSimulations.RenewableFullDispatch
      Loads:
        device_type = PowerSystems.PowerLoad
        formulation = PowerSimulations.StaticPowerLoad
      RenFx:
        device_type = PowerSystems.RenewableFix
        formulation = PowerSimulations.RenewableFixed
  branches: 
      T:
        device_type = PowerSystems.Transformer2W
        formulation = PowerSimulations.StaticTransformer
      TT:
        device_type = Po

### 5-Minute system
We had already created a `sys::System` from hourly RTS data in the OperationsProblem example.
The RTS data also includes 5-minute resolution time series data. So, we can create another
`System`:

In [2]:
sys_RT = System(rawsys; forecast_resolution = Dates.Minute(5))

┌ Info: Adding contributing generators for Spin_Up_R1 by category
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/m5IPy/src/parsers/power_system_table_data.jl:643
┌ Info: Adding contributing generators for Spin_Up_R2 by category
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/m5IPy/src/parsers/power_system_table_data.jl:643
┌ Info: Adding contributing generators for Spin_Up_R3 by category
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/m5IPy/src/parsers/power_system_table_data.jl:643
┌ Info: Adding contributing generators for Flex_Up by category
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/m5IPy/src/parsers/power_system_table_data.jl:643
┌ Info: Adding contributing generators for Flex_Down by category
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/m5IPy/src/parsers/power_system_table_data.jl:643
┌ Info: Adding contributing generators for Reg_Up by category
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSys

,ConcreteType,SuperTypes,Count
,String,String,Int64
1,Area,AggregationTopology <: Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,3
2,Bus,Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,73
3,GenericBattery,Storage <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,1
4,HVDCLine,DCBranch <: Branch <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,1
5,HydroDispatch,HydroGen <: Generator <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,20
6,Line,ACBranch <: Branch <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,104
7,LoadZone,AggregationTopology <: Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,21
8,PowerLoad,StaticLoad <: ElectricLoad <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,51
9,RenewableDispatch,RenewableGen <: Generator <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,30


The RTS dataset has some large RE and load forecast errors that create infeasible
simulations. Until we enable formulations with slack variables, we need to increase the
amount of reserve held to handle the forecast error.

In [3]:
get_component(VariableReserve{ReserveUp}, sys, "Flex_Up").requirement = 5.0

5.0

## `OperationsProblemTemplate`s define `Stage`s
Sequential simulations in PowerSimulations are created by defining `OperationsProblems`
that represent `Stages`, and how information flows between executions of a `Stage` and
between different `Stage`s.

Let's start by defining a two stage simulation that might look like a typical day-Ahead
and real-time electricity market clearing process.

### Define the reference model for the day-ahead unit commitment

In [4]:
devices = Dict(
    :Generators => DeviceModel(ThermalStandard, ThermalStandardUnitCommitment),
    :Ren => DeviceModel(RenewableDispatch, RenewableFullDispatch),
    :Loads => DeviceModel(PowerLoad, StaticPowerLoad),
    :HydroROR => DeviceModel(HydroDispatch, HydroDispatchRunOfRiver),
    :RenFx => DeviceModel(RenewableFix, RenewableFixed),
)
template_uc = template_unit_commitment(devices = devices)


Operations Problem Specification

  transmission:  PowerSimulations.CopperPlatePowerModel
  devices: 
      HydroROR:
        device_type = PowerSystems.HydroDispatch
        formulation = PowerSimulations.HydroDispatchRunOfRiver
      Generators:
        device_type = PowerSystems.ThermalStandard
        formulation = PowerSimulations.ThermalStandardUnitCommitment
      Ren:
        device_type = PowerSystems.RenewableDispatch
        formulation = PowerSimulations.RenewableFullDispatch
      Loads:
        device_type = PowerSystems.PowerLoad
        formulation = PowerSimulations.StaticPowerLoad
      RenFx:
        device_type = PowerSystems.RenewableFix
        formulation = PowerSimulations.RenewableFixed
  branches: 
      T:
        device_type = PowerSystems.Transformer2W
        formulation = PowerSimulations.StaticTransformer
      TT:
        device_type = PowerSystems.TapTransformer
        formulation = PowerSimulations.StaticTransformer
      L:
        device_type = Po

### Define the reference model for the real-time economic dispatch

In [5]:
devices = Dict(
    :Generators => DeviceModel(ThermalStandard, ThermalDispatch),
    :Ren => DeviceModel(RenewableDispatch, RenewableFullDispatch),
    :Loads => DeviceModel(PowerLoad, StaticPowerLoad),
    :HydroROR => DeviceModel(HydroDispatch, HydroDispatchRunOfRiver),
    :RenFx => DeviceModel(RenewableFix, RenewableFixed),
)
template_ed = template_economic_dispatch(devices = devices)


Operations Problem Specification

  transmission:  PowerSimulations.CopperPlatePowerModel
  devices: 
      HydroROR:
        device_type = PowerSystems.HydroDispatch
        formulation = PowerSimulations.HydroDispatchRunOfRiver
      Generators:
        device_type = PowerSystems.ThermalStandard
        formulation = PowerSimulations.ThermalDispatch
      Ren:
        device_type = PowerSystems.RenewableDispatch
        formulation = PowerSimulations.RenewableFullDispatch
      Loads:
        device_type = PowerSystems.PowerLoad
        formulation = PowerSimulations.StaticPowerLoad
      RenFx:
        device_type = PowerSystems.RenewableFix
        formulation = PowerSimulations.RenewableFixed
  branches: 
      T:
        device_type = PowerSystems.Transformer2W
        formulation = PowerSimulations.StaticTransformer
      TT:
        device_type = PowerSystems.TapTransformer
        formulation = PowerSimulations.StaticTransformer
      L:
        device_type = PowerSystems.Lin

### Define the `Stage`s
Stages define models. The actual problem will change as the stage gets updated to represent
different time periods, but the formulations applied to the components is constant within
a stage. In this case, we want to define two stages with the `OperationsProblemTemplate`s
and the `System`s that we've already created.

In [6]:
stages_definition = Dict(
    "UC" => Stage(GenericOpProblem, template_uc, sys, solver),
    "ED" => Stage(GenericOpProblem, template_ed, sys_RT, solver),
)

Dict{String,PowerSimulations.Stage{PowerSimulations.GenericOpProblem}} with 2 entries:
  "ED" => Stage()…
  "UC" => Stage()…

### `SimulationSequence`
Similar to an `OperationsProblemTemplate`, the `SimulationSequence` provides a template of
how to execute a sequential set of operations problems.

print_struct(SimulationSequence)

Let's review some of the `SimulationSequence` arguments.

### Chronologies
In PowerSimulations, chronologies define where information is flowing. There are two types
of chronologies.
 - inter-stage chronologies: Define how information flows between stages. e.g. day-ahead
solutions are used to inform economic dispatch problems
 - intra-stage chronologies: Define how information flows between multiple executions of a
single stage. e.g. the dispatch setpoints of the first period of an economic dispatch problem
are constrained by the ramping limits from setpoints in the final period of the previous problem.

Let's define an inter-stage chronolgy that synchronizes information from 24 periods of
the first stage with a set of executions of the second stage:

In [7]:
feedforward_chronologies = Dict(("UC" => "ED") => Synchronize(periods = 24))

Dict{Pair{String,String},PowerSimulations.Synchronize} with 1 entry:
  "UC"=>"ED" => Synchronize(24)

### `FeedForward` and `Cache`
The definition of exactly what information is passed using the defined chronologies is
accomplished with `FeedForward` and `Cache` objects. Specifically, `FeedForward` is used
to define what to do with information being passed with an inter-stage chronology. Let's
define a `FeedForward` that affects the semi-continuous range constraints of thermal generators
in the economic dispatch problems based on the value of the unit-commitment variables.

In [8]:
feedforward = Dict(
    ("ED", :devices, :Generators) => SemiContinuousFF(
        binary_from_stage = PSI.ON,
        affected_variables = [PSI.ACTIVE_POWER],
    ),
)

Dict{Tuple{String,Symbol,Symbol},PowerSimulations.SemiContinuousFF} with 1 entry:
  ("ED", :devices, :Generators) => SemiContinuousFF(:On, Symbol[:P], nothing)

The `Cache` is simply a way to preserve needed information for later use. In the case of
a typical day-ahead - real-time market simulation, there are many economic dispatch executions
in between each unit-commitment execution. Rather than keeping the full set of results from
previous unit-commitment simulations in memory to be used in later executions, we can define
exactly which results will be needed and carry them through a cache in the economic dispatch
problems for later use.

In [9]:
cache = Dict(("UC",) => TimeStatusChange(PSY.ThermalStandard, PSI.ON))

Dict{Tuple{String},PowerSimulations.TimeStatusChange} with 1 entry:
  ("UC",) => TimeStatusChange(ThermalStandard, 1-dimensional DenseAxisArray{Dic…

### Sequencing
The stage problem length, look-ahead, and other details surrounding the temporal Sequencing
of stages are controlled using the `order`, `horizons`, and `intervals` arguments.
 - order::Dict(Int, String) : the hierarchical order of stages in the simulation
 - horizons::Dict(String, Int) : defines the number of time periods in each stage (problem length)
 - intervals::Dict(String, Dates.Period) : defines the interval with which stage problems
advance after each execution. e.g. day-ahead problems have an interval of 24-hours

So, to define a typical day-ahead - real-time sequence, we can define the following:
 - Day ahead problems should represent 48 hours, advancing 24 hours after each execution (24-hour look-ahead)
 - Real time problems should represent 1 hour (12 5-minute periods), advancing 1 hour after each execution (no look-ahead)

In [10]:
order = Dict(1 => "UC", 2 => "ED")
horizons = Dict("UC" => 24, "ED" => 12)
intervals = Dict("UC" => (Hour(24), Consecutive()), "ED" => (Hour(1), Consecutive()))

Dict{String,Tuple{Dates.Hour,PowerSimulations.Consecutive}} with 2 entries:
  "ED" => (1 hour, Consecutive())
  "UC" => (24 hours, Consecutive())

Finally, we can put it all together:

In [11]:
DA_RT_sequence = SimulationSequence(
    step_resolution = Hour(24),
    order = order,
    horizons = horizons,
    intervals = intervals,
    ini_cond_chronology = InterStageChronology(),
    feedforward_chronologies = feedforward_chronologies,
    feedforward = feedforward,
    cache = cache,
)

Feed Forward Chronology
-----------------------

ED: PowerSimulations.SemiContinuousFF -> Generators

                     UC--┐ from : On
                         |
┌----┬----┬----┬----┬----┼----┬----┬----┬----┬----┬----┐
|    |    |    |    |    |    |    |    |    |    |    |
|    |    |    |    |    |    |    |    |    |    |    |
└─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED ... (x24) to : ["P"]

Initial Condition Chronology
----------------------------

1
|
|
2 --> 2 ... (x24)   


## `Simulation`
Now, we can build and execute a simulation using the `SimulationSequence` and `Stage`s
that we've defined.

In [12]:
sim = Simulation(
    name = "rts-test",
    steps = 1,
    stages = stages_definition,
    stages_sequence = DA_RT_sequence,
    simulation_folder = rts_dir,
)

Simulation()


### Build simulation

In [13]:
build!(sim)

### Execute simulation

In [14]:
sim_results = execute!(sim)

Executing Step 1
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Oct  7 2019 

command line - Cbc_C_Interface -ratioGap 0.5 -logLevel 1 -solve -quit (default strategy 1)
ratioGap was changed from 0 to 0.5
Continuous objective value is 1.09972e+06 - 0.33 seconds
Cgl0003I 46 fixed, 0 tightened bounds, 1150 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 48 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1104 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1039 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 934 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 428 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 406 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 372 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 361 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened

Simulation Results Reference

Results Folder: /Users/cbarrows/Documents/repos/SIIPExamples.jl/RTS-GMLC-master/rts-test/4/results

Reference Tables

stage-UC

Flex_Up__PowerSystems.VariableReserve{PowerSystems.ReserveUp}: dataframe size (1, 3)

parameter_P_PowerSystems.PowerLoad: dataframe size (1, 3)

parameter_P_PowerSystems.RenewableDispatch: dataframe size (1, 3)

Spin_Up_R1__PowerSystems.VariableReserve{PowerSystems.ReserveUp}: dataframe size (1, 3)

parameter_service_requirement_PowerSystems.VariableReserve{PowerSystems.ReserveUp}: dataframe size (1, 3)

Spin_Up_R3__PowerSystems.VariableReserve{PowerSystems.ReserveUp}: dataframe size (1, 3)

parameter_P_PowerSystems.RenewableFix: dataframe size (1, 3)

Spin_Up_R2__PowerSystems.VariableReserve{PowerSystems.ReserveUp}: dataframe size (1, 3)

Flex_Down__PowerSystems.VariableReserve{PowerSystems.ReserveDown}: dataframe size (1, 3)

Stop__PowerSystems.ThermalStandard: dataframe size (1, 3)

P__PowerSystems.ThermalStandard: dataframe size (1, 3)

P__PowerSystems.RenewableDispatch: dataframe size (1, 3)

Reg_Up__PowerSystems.VariableReserve{PowerSystems.ReserveUp}: dataframe size (1, 3)

P__PowerSystems.HydroDispatch: dataframe size (1, 3)

On__PowerSystems.ThermalStandard: dataframe size (1, 3)

Reg_Down__PowerSystems.VariableReserve{PowerSystems.ReserveDown}: dataframe size (1, 3)

parameter_P_PowerSystems.HydroDispatch: dataframe size (1, 3)

Start__PowerSystems.ThermalStandard: dataframe size (1, 3)

parameter_service_requirement_PowerSystems.VariableReserve{PowerSystems.ReserveDown}: dataframe size (1, 3)

stage-ED

parameter_P_PowerSystems.RenewableFix: dataframe size (24, 3)

P__PowerSystems.HydroDispatch: dataframe size (24, 3)

parameter_P_PowerSystems.PowerLoad: dataframe size (24, 3)

parameter_P_PowerSystems.RenewableDispatch: dataframe size (24, 3)

P__PowerSystems.ThermalStandard: dataframe size (24, 3)

P__PowerSystems.RenewableDispatch: dataframe size (24, 3)

parameter_P_PowerSystems.HydroDispatch: dataframe size (24, 3)

stage-UC

time length: 24

stage-ED

time length: 12

## Results

In [15]:
uc_results = load_simulation_results(sim_results, "UC");
ed_results = load_simulation_results(sim_results, "ED");

## Plotting
Take a look at the examples in [the plotting folder.](../../notebook/PowerSimulations_examples/Plotting)

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*